# Copy Datasets Using Globus


Each attendee has access to their own Guest Collection, which is a shared endpoint that can be used to host and share data.


**TODO**

* Explain how to login
* Explain how to find the Guest Collection

Specify the UUID of your Guest Collection in `my_collection_uuid`

For this tutorial, we assume you're putting all of the datasets under a single folder. This isn't necessary, it just makes things easier.

Define your collection's base folder using the `my_collection_folder` variable. Reasonable names include "datasets", "data", or "repository". These forlder names are just for hierachical organization.

In [ ]:
%cd cheapandfair-template/

In [8]:
import globus_utils

In [ ]:
%%file config.toml
# The following refer to the destination collection, where the data will be copied to and that will serve as backend for the data portal
UUID='18ed636e-0389-44c3-b533-cb3901dfc60f'
FOLDER='/myfolder3/'
DOMAIN='g-1926f5.c2d0f8.bd7c.data.globus.org'
# collection of the source data, during the tutorial this is the "Cheap and FAIR Tutorial Datasets" collection
SOURCE_UUID='7352d991-b0a0-49a2-830c-e8fe8c968ca2'
SOURCE_FOLDER='/public/datasets/'

We copy the dataset by providing the name of the dataset, the UUID of the destination Guest Collection, and the folder we want it copied to.

The method will return the base URL of the Guest Collection and the manifest of the files that were copied. The URL returned will be the same for each of the datasets we copy because they're all going to the same Guest Collection. The method will also write a copy of the manifest to the local directory named `<dataset>-manifest.json`.

## Authenticate with Globus

Login to Globus using the following command. This will open a browser window where you can login to Globus and receive a token to paste back into the notebook.

In [ ]:
copy_dataset.login();

## Copy Datasets and create manifests

Now we can copy the datasets from the source to the destination collection.
This script will also create `manifests.json` files which contain the list of files and their checksums.

In [ ]:
cmb_manifest = copy_dataset.copydataset('cmb')

Let's look at a couple of the entries in the file manifest.

In [ ]:
print(json.dumps(cmb_manifest[:2], indent=2))

Now we can copy the other two datasets. You won't need to login again because the tokens have been cached in `~/.cheapandfair.json`.

In [ ]:
dust_manifest = copy_dataset.copydataset('dust')
synch_manifest = copy_dataset.copydataset('synch')

We can see that the manifests were saved locally.

In [ ]:
!ls *.json

TODO now that we have the datasets copied

* Explain how to configure permissions on each folder
